In [1]:
# Example and test bed for the constant salt ensemble
import numpy as np
from numpy import random
from simtk import openmm, unit
from simtk.openmm import app
import sys
from datetime import datetime

from sys import stdout
sys.path.append("../saltswap/")
import saltswap

kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA

In [2]:
#Loading a premade water box:
temperature = 300*unit.kelvin
pdb = app.PDBFile('../examples/waterbox.pdb')
forcefield = app.ForceField('tip3p.xml')
system = forcefield.createSystem(pdb.topology,nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds)
integrator = openmm.LangevinIntegrator(300*unit.kelvin, 1/unit.picosecond, 0.002*unit.picoseconds)
system.addForce(openmm.MonteCarloBarostat(1*unit.atmospheres, 300*unit.kelvin, 25))
simulation = app.Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)

In [3]:
# Defining the various parameters of the constant salt simulation:
integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
nattempts_per_update = 10
iterations = 100       # Number of rounds of MD and constant salt moves
nsteps = 100           # Amount of MD steps per iteration (not to be confused with number of steps in NCMC)
nattempts = 1000        # Number of identity exchanges for water and ions.
delta_chem = -160*unit.kilojoule_per_mole       # The free energy penalty to turn 2 waters into Na and Cl. To be calibrated.


In [5]:
# Initialing constant salt class.
ss = saltswap.SaltSwap(system=system,topology=pdb.topology,temperature=temperature,
                   delta_chem=delta_chem,integrator=integrator,debug=False)
        

In [14]:
# Timing new fractional update function.
iters = 100000
startTime = datetime.now()
for i in range(iters):
    ss.updateForces_fractional(mode="add salt",exchange_indices=np.array([5,10]),fraction=0.5)
tm_old = datetime.now() - startTime
print "New script:", tm_old.seconds

startTime = datetime.now()
for i in range(iters):
    ss.updateForces(mode="add salt",exchange_indices=np.array([5,10]))
tm_new = datetime.now() - startTime
print "Old script:", tm_new.seconds

print "fraction increase in time:",1.0*tm_new.seconds/tm_old.seconds

New script: 46
Old script: 17
fraction increase in time: 0.369565217391


In [23]:
ss.update(simulation.context)

In [16]:
ss.getAcceptanceProbability()

0.46

In [13]:
print "fraction increase in time:",1.0*tm_new.seconds/tm_old.seconds

fraction increase in time: 0.0212765957447


In [16]:
46.0/17

2.7058823529411766

In [21]:
46.0/iters*100*1000*10/60

7.666666666666667

In [22]:
cat2wat = []*3

In [29]:
m = ss.mutable_residues[21]
m.name
num_na = 0 
for res in ss.mutable_residues:
    if res.name == "Na+":
        num_na += 1


In [30]:
num_na

7

In [33]:
molecule = [atom for atom in ss.mutable_residues[21].atoms()]

In [34]:
m=molecule[0]

[<Atom 63 (O) of chain 0 residue 21 (Na+)>,
 <Atom 64 (H1) of chain 0 residue 21 (Na+)>,
 <Atom 65 (H2) of chain 0 residue 21 (Na+)>]

In [6]:
iters = 1000
startTime = datetime.now()
for i in range(iters):
    ss.updateForces_fractional(mode="remove salt",exchange_indices=np.array([5,10]),fraction=0.5)
tm_old = datetime.now() - startTime
print "New script:", tm_old.seconds

New script: 0


In [8]:
nkernals = 1
for k in range(nkernals):
    fraction = float(k + 1)/float(nkernals)
    print fraction

1.0
